<a href="https://colab.research.google.com/drive/1jFJ7l7jIv22BhvvzlmXOWFtgBE15ea2X" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install requirements

In [1]:
!pip install git+https://github.com/kovanostra/message-passing-nn/
!pip install pytorch==1.4.0 pandas==1.0.3

  Cloning https://github.com/kovanostra/message-passing-nn/ to /tmp/pip-req-build-yw3tqdnl
  Running command git clone -q https://github.com/kovanostra/message-passing-nn/ /tmp/pip-req-build-yw3tqdnl
     |████████████████████████████████| 20.0MB 1.3MB/s 
     |████████████████████████████████| 753.4MB 21kB/s 
  Created wheel for message-passing-nn: filename=message_passing_nn-1.0-cp36-none-any.whl size=28731 sha256=008013490ce2f521d4cb228537654992b7578929fbe476f901eefbb23f49b67a
  Stored in directory: /tmp/pip-ephem-wheel-cache-n0j4pojg/wheels/c0/3f/90/55c69f6e0e88546e9fc867544ae0beabab39968d44f0ffc2ea
Successfully built message-passing-nn
ERROR: torchvision 0.6.0+cu101 has requirement torch==1.5.0, but you'll have torch 1.4.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found exi

ERROR: Could not find a version that satisfies the requirement pytorch==1.4.0 (from versions: 0.1.2, 1.0.2)
ERROR: No matching distribution found for pytorch==1.4.0


## Imports

In [0]:
import torch
from src.domain.model_trainer import ModelTrainer
from src.domain.graph_encoder import GraphEncoder
from src.domain.graph import Graph
from src.domain.data_preprocessor import DataPreprocessor
from src.repository.training_data_repository import TrainingDataRepository

## Set up the variables 

In [0]:
device = 'cuda'
epochs = 10
loss_function = 'MSE'
optimizer = 'SGD'
batch_size = 1
validation_split = 0.2
test_split = 0.1
time_steps = 5
validation_period = 5

dataset_size = 10
number_of_nodes = 10
number_of_node_features = 2
fully_connected_layer_input_size = number_of_nodes*number_of_node_features
fully_connected_layer_output_size = number_of_nodes**2

configuration_dictionary = {'time_steps': time_steps,
                            'loss_function': loss_function,
                            'optimizer': optimizer}

## Set up the dataset.

In [0]:
dataset_name = 'sample-dataset'
data_directory = 'data/'
training_data_repository = TrainingDataRepository(data_directory, dataset_name)
raw_dataset = training_data_repository.get_all_features_and_labels_from_separate_files()
initialization_graph = DataPreprocessor.extract_initialization_graph(raw_dataset)
training_data, validation_data, test_data = DataPreprocessor.train_validation_test_split(raw_dataset, 
                                                                                         batch_size, 
                                                                                         validation_split, 
                                                                                         test_split)

## Instatiate the model and the model trainer

In [0]:
graph_encoder = GraphEncoder(time_steps, 
                             number_of_nodes, 
                             number_of_node_features, 
                             fully_connected_layer_input_size, 
                             fully_connected_layer_output_size)

model_trainer = ModelTrainer(GraphEncoder, device)
model_trainer.instantiate_attributes(initialization_graph, configuration_dictionary)

## Train the model

In [7]:
for epoch in range(epochs):
    training_loss = model_trainer.do_train(training_data, epoch)
    print("Epoch", epoch, "Training loss:", training_loss)
    if epoch % validation_period == 0:
        validation_loss = model_trainer.do_evaluate(validation_data, epoch)
        print("Epoch", epoch, "Validation loss:", validation_loss)
test_loss = model_trainer.do_evaluate(test_data)
print("Test loss:", validation_loss)

RuntimeError: ignored